In [1]:
import tensorflow as tf
import os
import numpy as np

In [2]:
base_dir=r"/content/drive/MyDrive/Thesis Dataset"

In [3]:
# Pre-processing Step
IMAGE_SIZE=224
BATCH_SIZE=32
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)
validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1)

In [4]:
train_generator=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)
validation_generator=validation_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 4078 images belonging to 4 classes.
Found 406 images belonging to 4 classes.


In [5]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob

In [6]:
IMAGE_SIZE=[224,224]
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [7]:
for layer in vgg.layers:
  layer.trainable=False

In [8]:
folders=glob("/content/drive/MyDrive/Thesis Dataset/*")
print(len(folders))

4


In [9]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
epoch=50
history=model.fit(train_generator,
                  steps_per_epoch=len(train_generator),
                  epochs=epoch,
                  validation_data=validation_generator,
                  validation_steps=len(validation_generator)
                  )

Epoch 1/50
128/128 [==============================] - 270s 2s/step - loss: 0.3701 - accuracy: 0.6366 - val_loss: 0.3850 - val_accuracy: 0.5616
Epoch 2/50
128/128 [==============================] - 247s 2s/step - loss: 0.3040 - accuracy: 0.6923 - val_loss: 0.3428 - val_accuracy: 0.6552
Epoch 3/50
128/128 [==============================] - 266s 2s/step - loss: 0.2775 - accuracy: 0.7369 - val_loss: 0.3742 - val_accuracy: 0.6429
Epoch 4/50
128/128 [==============================] - 248s 2s/step - loss: 0.2735 - accuracy: 0.7413 - val_loss: 0.3622 - val_accuracy: 0.6379
Epoch 5/50
128/128 [==============================] - 250s 2s/step - loss: 0.2632 - accuracy: 0.7553 - val_loss: 0.3002 - val_accuracy: 0.7069
Epoch 6/50
128/128 [==============================] - 248s 2s/step - loss: 0.2523 - accuracy: 0.7685 - val_loss: 0.3207 - val_accuracy: 0.7044
Epoch 7/50
128/128 [==============================] - 246s 2s/step - loss: 0.2341 - accuracy: 0.7935 - val_loss: 0.3744 - val_accuracy: 0.6527

In [ ]:
# display the loss and accuracy curves

import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='T raining loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()


plt.plot(epochs, acc, 'bo', label='T raining acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.show()

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

In [ ]:
# Save the entire model as a SavedModel.
# Saving the model for Future Inferences

model_json = model.to_json()
with open("vgg-16.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5

tf.keras.models.save_model(model,"vgg-16.h5")

In [17]:
# imports

from keras.models import model_from_json 

# opening and store file in a variable

json_file = open('vgg-16.json','r')
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model

loaded_model = model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights("vgg-16.h5")
print("Loaded Model from disk")

# compile and evaluate loaded model

loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


Loaded Model from disk


In [18]:
conf_matrix = tf.math.confusion_matrix(train_generator, validation_generator)
print(conf_matrix) 

ValueError: ignored